In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from nltk.tokenize import sent_tokenize
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx

In [27]:
pip install wikipedia-api

In [26]:
T1 = open('wikipedia.py', mode='r', encoding='UTF-8')

In [2]:
def get_only_text(url):
    """
    return the title and the text of the article
    at the specified url
    """
    page = urlopen(url)
    soup = BeautifulSoup(page, "lxml")
    text = ' '.join(map(lambda p: p.text, soup.find_all('p')))

    print ("=====================")
    print (text)
    print ("=====================")

    return soup.title.text, text

In [3]:
url="https://www.whitehouse.gov/briefing-room/statements-releases/2023/11/09/statement-from-president-joe-biden-on-senator-joe-manchin/"
text = get_only_text(url)
sentences = []
for s in text:
    sentences.append(sent_tokenize(s))

sentences = [y for x in sentences for y in x]
sentences[30:40]


The White House
								1600 Pennsylvania Ave NW
								Washington, DC 20500
							 For more than forty years – as a state legislator, a Secretary of State, a Governor, and a Senator – Joe Manchin has dedicated himself to serving the people of his beloved West Virginia. During my time as Vice President and now as President, Joe and I have worked together to get things done for hardworking families. From the Bipartisan Infrastructure Law – which is upgrading America’s aging infrastructure, to the PACT Act – which is making sure our veterans get the care they deserve, to the Inflation Reduction Act – which is strengthening our energy security and lowering prescription drug costs for our seniors, to the most meaningful gun safety legislation in three decades, we’ve made real progress. I was also proud when Joe voted to make history and confirm Ketanji Brown Jackson as the first Black woman to serve on the United States Supreme Court. Joe, Gayle, and the entire Manchin family should fee

[]

In [4]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [5]:
len(word_embeddings)

400000

In [6]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

<ipython-input-6-a477335c5891>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")


In [7]:
# change to lowercase
import nltk
nltk.download('stopwords')
clean_sentences = [s.lower() for s in clean_sentences]
stop_words = stopwords.words('english')
# function to remove stopwords
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
sentence_vectors = []
for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    sentence_vectors.append(v)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
# Create an empty similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [9]:
# Extract top 15 sentences as the summary representation
for i in range(10):
    print(ranked_sentences[i][1])

### We'll be in touch with the latest information on how President Biden and his administration are working for the American people, as well as ways you can get involved and help our country build back better.
During my time as Vice President and now as President, Joe and I have worked together to get things done for hardworking families.
I was also proud when Joe voted to make history and confirm Ketanji Brown Jackson as the first Black woman to serve on the United States Supreme Court.

The White House
								1600 Pennsylvania Ave NW
								Washington, DC 20500
							 For more than forty years – as a state legislator, a Secretary of State, a Governor, and a Senator – Joe Manchin has dedicated himself to serving the people of his beloved West Virginia.
Joe, Gayle, and the entire Manchin family should feel proud of the Senator’s service to West Virginia and to our country.
I look forward to continuing our work together to get things done for the American people.
From the Bipartisan 

In [10]:
def get_only_text(url):
    """
    return the title and the text of the article
    at the specified url
    """
    page = urlopen(url)
    soup = BeautifulSoup(page, "lxml")
    text = ' '.join(map(lambda p: p.text, soup.find_all('p')))

    print ("=====================")
    print (text)
    print ("=====================")

    return soup.title.text, text

In [11]:
url="https://www.whitehouse.gov/briefing-room/statements-releases/2023/11/09/statement-from-president-joe-biden-on-senator-joe-manchin/"
text = get_only_text(url)


The White House
								1600 Pennsylvania Ave NW
								Washington, DC 20500
							 For more than forty years – as a state legislator, a Secretary of State, a Governor, and a Senator – Joe Manchin has dedicated himself to serving the people of his beloved West Virginia. During my time as Vice President and now as President, Joe and I have worked together to get things done for hardworking families. From the Bipartisan Infrastructure Law – which is upgrading America’s aging infrastructure, to the PACT Act – which is making sure our veterans get the care they deserve, to the Inflation Reduction Act – which is strengthening our energy security and lowering prescription drug costs for our seniors, to the most meaningful gun safety legislation in three decades, we’ve made real progress. I was also proud when Joe voted to make history and confirm Ketanji Brown Jackson as the first Black woman to serve on the United States Supreme Court. Joe, Gayle, and the entire Manchin family should fee

In [12]:
sentences = []
for s in text:
    sentences.append(sent_tokenize(s))

sentences = [y for x in sentences for y in x]
sentences[30:40]

[]

In [13]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [14]:
len(word_embeddings)

400000

In [15]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

<ipython-input-15-a477335c5891>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")


In [16]:
# change to lowercase
clean_sentences = [s.lower() for s in clean_sentences]
stop_words = stopwords.words('english')
# function to remove stopwords
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
sentence_vectors = []
for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    sentence_vectors.append(v)
# Create an empty similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
# Extract top 15 sentences as the summary representation
for i in range(10):
    print(ranked_sentences[i][1])

### We'll be in touch with the latest information on how President Biden and his administration are working for the American people, as well as ways you can get involved and help our country build back better.
During my time as Vice President and now as President, Joe and I have worked together to get things done for hardworking families.
I was also proud when Joe voted to make history and confirm Ketanji Brown Jackson as the first Black woman to serve on the United States Supreme Court.

The White House
								1600 Pennsylvania Ave NW
								Washington, DC 20500
							 For more than forty years – as a state legislator, a Secretary of State, a Governor, and a Senator – Joe Manchin has dedicated himself to serving the people of his beloved West Virginia.
Joe, Gayle, and the entire Manchin family should feel proud of the Senator’s service to West Virginia and to our country.
I look forward to continuing our work together to get things done for the American people.
From the Bipartisan 